In [1]:
import matplotlib.pyplot as plt

import pyBeamSim
import numpy as np
import random
import time
import pandas as pd
import torch
import torch.nn as nn

from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from model.old_model import Encoder_Decoder, MLP, MLP2
import os

In [2]:
df = pd.read_csv('../data/data_10000.csv', index_col=0)
df = df[:10000]
df

,drift1_len,quad1_len,quad1_gra,drift2_len,quad2_len,quad2_gra,x_avg0,x_avg1,x_avg2,x_avg3,x_avg4
0,0.402310,0.450841,-16.332723,0.170125,0.213844,7.026671,-2.582750e-09,0.000043,-1.950323e-07,-0.000093,0.000375
1,0.237463,0.454075,14.937412,0.273825,0.334553,5.617945,-2.582750e-09,0.000027,-4.348649e-06,-0.000777,-0.000208
2,0.414345,0.177982,19.552067,0.388376,0.442580,2.004069,-2.582750e-09,0.000037,-8.071600e-07,0.000113,-0.000061
3,0.246283,0.124380,0.320149,0.130190,0.210073,13.761691,-2.582750e-09,0.000028,-4.777946e-06,-0.000069,-0.000029
4,0.256067,0.194462,6.097415,0.473669,0.432795,7.627331,-2.582750e-09,0.000029,-3.698168e-06,-0.000044,-0.000005
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.247073,0.286787,2.478341,0.375353,0.320700,-13.608154,-2.582750e-09,0.000028,2.053298e-06,0.000054,-0.000061
9996,0.407613,0.432325,4.456587,0.302943,0.444821,4.182542,-2.582750e-09,0.000039,-3.394178e-07,0.000015,-0.000064
9997,0.148527,0.189331,10.690400,0.205502,0.385139,9.790066,-2.582750e-09,0.000017,1.513054e-06,0.000009,-0.000010
9998,0.307523,0.318801,2.121998,0.348172,0.262224,-10.475280,-2.582750e-09,0.000031,-4.560829e-07,-0.000015,-0.000034


In [3]:
# use scaler
scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
change_columns = df.columns[-4:]
df[change_columns] = scaler2.fit_transform(df[change_columns])
df

,drift1_len,quad1_len,quad1_gra,drift2_len,quad2_len,quad2_gra,x_avg0,x_avg1,x_avg2,x_avg3,x_avg4
0,0.402310,0.450841,-16.332723,0.170125,0.213844,7.026671,-2.582750e-09,1.792541,0.179957,-0.004260,2.760523
1,0.237463,0.454075,14.937412,0.273825,0.334553,5.617945,-2.582750e-09,-0.127419,-1.718419,-2.768817,-1.031767
2,0.414345,0.177982,19.552067,0.388376,0.442580,2.004069,-2.582750e-09,1.094240,-0.099811,0.825994,-0.078466
3,0.246283,0.124380,0.320149,0.130190,0.210073,13.761691,-2.582750e-09,-0.005783,-1.914625,0.090404,0.134357
4,0.256067,0.194462,6.097415,0.473669,0.432795,7.627331,-2.582750e-09,0.129143,-1.421122,0.193127,0.286404
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.247073,0.286787,2.478341,0.375353,0.320700,-13.608154,-2.582750e-09,0.005115,1.207537,0.586923,-0.076678
9996,0.407613,0.432325,4.456587,0.302943,0.444821,4.182542,-2.582750e-09,1.330964,0.113967,0.433091,-0.095317
9997,0.148527,0.189331,10.690400,0.205502,0.385139,9.790066,-2.582750e-09,-1.353881,0.960624,0.408484,0.254405
9998,0.307523,0.318801,2.121998,0.348172,0.262224,-10.475280,-2.582750e-09,0.414870,0.060646,0.310102,0.100083


In [4]:
# last 4 columns as y_true
X = df.iloc[:, :-5]
y = df.iloc[:, -4:]
# transform df into tensor
X = torch.Tensor(X.values)
y = torch.Tensor(y.values)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

print(f'X_train.shape: {X_train.shape}')
print(f'y_train.shape: {y_train.shape}')

from torch.utils.data import TensorDataset, DataLoader

# generate dataset and dataloader
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)
    
first_y = train_dataset[0][1]
output_len = first_y.shape[0] - 1
print('the first y: ', first_y)

X_train.shape: torch.Size([8000, 6])
y_train.shape: torch.Size([8000, 4])
the first y:  tensor([-0.5333, -0.2510,  0.2763,  0.9040])


In [5]:
# use mean absolute percent error as criterion
# get mean mape for each batch
def get_mape(tensor_true, tensor_pred):
    """
    :param tensor_true : (batch_size, output_size) or (output_size,)
    :param tensor_pred : same shape as tensor_true
    """

    mape = torch.abs((tensor_pred - tensor_true) / tensor_true) * 100

    if tensor_true.dim() == 2:
        return torch.mean(mape, dim=1).mean()  
    else:
        return torch.mean(mape)  
# and R2 score

def count_all_parameters(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

y_true = torch.Tensor([100, 200, 300])
y_pred = torch.Tensor([110, 190, 310])
r2 = r2_score(y_true, y_pred)
mape = get_mape(y_true, y_pred)
print(f'R^2 Score: {r2:.2f}')
print(f'MAPE: {mape:.2f}%')

R^2 Score: 0.98
MAPE: 6.11%


In [6]:
input_size = 6
output_size = 4

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
mlp = MLP2(input_size=input_size, output_size=output_size)

mlp.to(device)

MLP2(
  (fc1): Linear(in_features=6, out_features=32, bias=True)
  (drop1): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (drop2): Dropout(p=0.1, inplace=False)
  (fc3): Linear(in_features=64, out_features=128, bias=True)
  (drop3): Dropout(p=0.1, inplace=False)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (drop4): Dropout(p=0.1, inplace=False)
  (fc5): Linear(in_features=64, out_features=32, bias=True)
  (drop5): Dropout(p=0.1, inplace=False)
  (output): Linear(in_features=32, out_features=4, bias=True)
)

In [6]:
num_epochs = 500
learning_rate = 0.01

optimizer = optim.Adam(mlp.parameters(), lr=learning_rate)

criterion = nn.MSELoss()

train_losses = []
test_losses = []
for epoch in range(num_epochs):
    if epoch == 0:
        print(f'The training is using {device}')
    
    # 训练模式
    mlp.train()
    total_loss = 0
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)
        y_pred = mlp(X)
        loss = criterion(y_pred, y)
        total_loss += loss.item()  # for each batch
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_loader)   # nums of batch
    train_losses.append(avg_train_loss)
    
    # for validate
    mlp.eval()
    total_val_loss = 0
    with torch.no_grad():  
        for X_val, y_val in test_loader:
            X_val = X_val.to(device)
            y_val = y_val.to(device)
            
            y_val_pred = mlp(X_val)
            val_loss = criterion(y_val_pred, y_val)
            total_val_loss += val_loss.item()
    
    # 计算并打印平均验证损失
    avg_val_loss = total_val_loss / len(test_loader)
    test_losses.append(avg_val_loss)
    
    print(f'Epoch {epoch+1}: train_loss: {avg_train_loss:.10f}, valid_loss: {avg_val_loss:.10f}')

The training is using cuda
Epoch 1: train_loss: 0.9982748013, valid_loss: 0.9906674959
Epoch 2: train_loss: 0.9924929166, valid_loss: 0.9882080778
Epoch 3: train_loss: 0.9885084338, valid_loss: 0.9793835394
Epoch 4: train_loss: 0.9676819024, valid_loss: 0.9412637409
Epoch 5: train_loss: 0.9452149282, valid_loss: 0.9048579037
Epoch 6: train_loss: 0.9223052764, valid_loss: 0.8804236893
Epoch 7: train_loss: 0.8662582049, valid_loss: 0.7919980381
Epoch 8: train_loss: 0.8029029970, valid_loss: 0.7029683413
Epoch 9: train_loss: 0.7561145267, valid_loss: 0.6821302865
Epoch 10: train_loss: 0.7554936724, valid_loss: 0.6566091757
Epoch 11: train_loss: 0.7170348279, valid_loss: 0.7193727586
Epoch 12: train_loss: 0.7213999784, valid_loss: 0.6262227800
Epoch 13: train_loss: 0.7124643776, valid_loss: 0.7799697146
Epoch 14: train_loss: 0.7897398791, valid_loss: 0.7432342581
Epoch 15: train_loss: 0.7609317484, valid_loss: 0.7534480710
Epoch 16: train_loss: 0.7196631618, valid_loss: 0.6343387524
Epoch 

In [7]:
save_pth = '../model_save'
model_pth = 'mlp' + '.pth'
torch.save(mlp.state_dict(), os.path.join(save_pth, model_pth))

save_pth2 = '../record_for_plot'
df_record = pd.DataFrame({
    'epoch': range(1, num_epochs + 1),
    'train_loss': train_losses,
    'test_loss': test_losses
})
loss_pth = 'mlp.csv'
df_record.to_csv(os.path.join(save_pth2, loss_pth), index=False)

In [7]:
def evaluate_test_mape_gpu(model, test_loader):
    model.eval()
    total_num = 0
    total_mape = 0
    with torch.no_grad():
        for X, y in test_loader:
            X = X.to('cuda')
            y = y.to('cuda')
            y_pred = model(X)
            mape = get_mape(y, y_pred) # avg for elem
            total_mape += mape
            total_num += X.shape[0]
    return total_mape / len(test_loader)

print(f'model has {count_all_parameters(mlp)} parameters')

model has 21124 parameters


In [8]:
mlp.load_state_dict(torch.load('../model_save/mlp.pth'))

print(evaluate_test_mape_gpu(mlp, test_loader))
print(evaluate_test_mape_gpu(mlp, train_loader))

tensor(255.2628, device='cuda:0')
tensor(250.5775, device='cuda:0')


In [10]:
"""
MAPE and loss

MLP

for 1000
Epoch 250: train_loss: 0.5033869537, valid_loss: 0.7438774705
test: 366.2326
train: 263.9838

for 10000
Epoch 250: train_loss: 0.5912603981, valid_loss: 0.5475129113
test: 255.2459
train: 352.6531


MLP2  10000 or 20000 params
for 1000
Epoch 500: train_loss: 0.6523829103, valid_loss: 0.4985363185
Epoch 500: train_loss: 0.5795483543, valid_loss: 0.8010688871
test: 271
train: 354

for 10000
Epoch 138: train_loss: 0.6780085616, valid_loss: 0.7711212859
test: 197
train: 293
"""

'\nMAPE and loss\n\nMLP\n\nfor 1000\nEpoch 250: train_loss: 0.5033869537, valid_loss: 0.7438774705\ntest: 366.2326\ntrain: 263.9838\n\nfor 10000\nEpoch 250: train_loss: 0.5912603981, valid_loss: 0.5475129113\ntest: 255.2459\ntrain: 352.6531\n\n\nMLP2  10000 or 20000 params\nfor 1000\nEpoch 500: train_loss: 0.6523829103, valid_loss: 0.4985363185\nEpoch 500: train_loss: 0.5795483543, valid_loss: 0.8010688871\ntest: 271\ntrain: 354\n\nfor 10000\nEpoch 138: train_loss: 0.6780085616, valid_loss: 0.7711212859\ntest: 197\ntrain: 293\n'

### evaluate test

In [26]:
def print_tensor(a: torch.Tensor, is_pred=True):
    list = a.squeeze().numpy().tolist()
    c = [round(b, 2) for b in list]
    if is_pred:
        print(c)
    else:
        print(c)
        
mlp.to('cpu')
num_train = random.randint(0, X_train.shape[0])
num_test = random.randint(0, X_test.shape[0])

train_input = train_dataset[num_train][0]
train_output = train_dataset[num_train][1]
test_input = test_dataset[num_test][0]
test_output = test_dataset[num_test][1]

mlp.eval()
with torch.no_grad():
    start_time = time.time()
    train_pred = mlp(train_input)
    end_time = time.time()
    test_pred = mlp(test_input)
    
consume_time = end_time - start_time
print(f'one predict time: {consume_time:.9f} seconds\n')
print('choose from train:')
print_tensor(train_output, False)
print_tensor(train_pred)
print(f'train_mape: {get_mape(train_output, train_pred)}\n')

print('choose from test:')
print_tensor(test_output, False)
print_tensor(test_pred)
print(f'test_mape: {get_mape(test_output, test_pred)}')

one predict time: 0.000000000 seconds

choose from train:
[1.86, 0.28, 0.71, 0.98]
[0.41, -0.06, 0.08, 0.06]
train_mape: 95.22486114501953

choose from test:
[0.24, -0.24, -0.35, -1.59]
[0.41, -0.06, 0.08, 0.06]
test_mape: 93.0389404296875
